In [ ]:
import numpy as np
import glob2
import datetime
from pathlib import Path
from tqdm.notebook import tqdm
import pickle
from matplotlib import pyplot as plt
from utils.detection.association_geodesic import squarize
import matplotlib as mpl
import matplotlib.dates as mdates
from scipy import stats
from utils.physics.geodesic.distance import distance_point_point

plt.style.use('classic')
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "text.usetex": True,
    "font.family": "serif",
    "font.size": 10,
    "axes.titlesize": 10,
    "axes.labelsize": 10,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "legend.fontsize": 8,
})
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
import math
from numpy.linalg import LinAlgError
import pandas as pd
from scipy.interpolate import RegularGridInterpolator

from utils.data_reading.sound_data.station import StationsCatalog
from utils.physics.sound_model.spherical_sound_model import GridSphericalSoundModel as GridSoundModel, MonthlyHomogeneousSphericalSoundModel as HomogeneousSoundModel
from utils.detection.association_geodesic import compute_candidates, update_valid_grid, update_results, load_detections, compute_grids

In [ ]:
plt.style.use('classic')
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "text.usetex": True,
    "font.family": "serif",
    "font.size": 10,
    "axes.titlesize": 10,
    "axes.labelsize": 10,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "legend.fontsize": 8,
})
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [ ]:
STATIONS = StationsCatalog("/media/plerolland/akoustik/MAHY").filter_out_undated().filter_out_unlocated()
seismic_paths = glob2.glob("../../../../data/MAHY/loc_3D/*.npz")
acoustic_to_s = {}
for s in STATIONS:
    depth, bathy = s.other_kwargs["depth"], s.other_kwargs["bathy"]
    under_hydro = bathy - depth
    acoustic_to_s[s] = under_hydro / 1520

interp_seismic = {}
for seismic_propa_path in seismic_paths:
    depth = float(seismic_propa_path.split("_")[-1].split("m")[0])
    data = np.load(seismic_propa_path)
    seismic_propagations = data["values"]
    seismic_propagations[(seismic_propagations < 10 ** -6) | (seismic_propagations > 10 ** 6)] = np.nan
    seismic_depths = data["depths"] * 1_000
    seismic_distances = data["distances"] * 1_000
    interp_seismic[depth] = RegularGridInterpolator(
        (seismic_depths, seismic_distances),
        seismic_propagations,
        bounds_error=False,  # allow extrapolation
        fill_value=None)
available_depths = np.array(list(interp_seismic.keys()))
s_to_interp = {s: interp_seismic[available_depths[np.argmin(abs(available_depths - s.other_kwargs["bathy"]))]]
               for s in STATIONS}

match_files = glob2.glob("../../../../data/MAHY/loc_3D/twin-cat/*_raw_OBS-fixed.csv")
match_files = glob2.glob("../../../../data/MAHY/loc_3D/twin-cat/*_raw.csv")
match = {}
for f in match_files:
    d = f.split("/")[-1].split("_")[0]
    match[d] = pd.read_csv(f, parse_dates=['date'] + [s.name for s in STATIONS.by_dataset(d)])

In [ ]:
r = np.sum([len(m) for m in match.values()])
print(r)

# TIMES

In [ ]:
deltas = {}

for dataset in match.keys():
    for idx in tqdm(match[dataset].index):
        a = match[dataset].loc[idx]
        for s in STATIONS.by_dataset(dataset):
            if not type(a[s.name]) == pd._libs.tslibs.nattype.NaTType:
                dist = distance_point_point([a["lat"],a["lon"]], s.get_pos())
                seismic_travel_path = s_to_interp[s]([[a["depth"]*1_000, dist]])[0]
                expected = seismic_travel_path + acoustic_to_s[s]
                actual = a[s.name] - a["date"]

                deltas.setdefault(s, {})[a["date"]] = actual.total_seconds() - expected

In [ ]:
# version with no fixed intercept
def dates_format(x, pos):
    dt = mdates.num2date(x)
    return r'\shortstack{%s\\%s}' % (dt.strftime('%d/%m'), dt.strftime('%Y'))

corrections = {}  # contains, for each station, the slope and intercept of a linear correction (starting from s.date_start)
res_csv = ""

for s in STATIONS:
    if s.dataset not in match:
        print(f"Skipping station {s}")
        continue

    values = np.array(list(deltas[s].values()))
    mask = np.abs(values - np.mean(values)) < 1*np.std(values)
    values = values[mask]
    times = np.array(list(deltas[s].keys()))[mask]
    values, times = values[np.argsort(times)], times[np.argsort(times)]
    t_sec = [(t - s.date_start).total_seconds() for t in times]

    (slope, intercept), cov = np.polyfit(t_sec, values, 1, cov=True)
    var_slope, var_offset = np.diag(cov)
    sigma_slope = np.sqrt(var_slope)
    sigma_offset = np.sqrt(var_offset)
    drift_ppm = slope * 1e6
    ci_slope_ppm = 1.96 * sigma_slope * 1e6  # 95% CI for slope in ppm

    #drift_fit = slope * np.array(t_sec) + intercept
    #residuals = values - drift_fit
    #diff_mean = values - np.mean(values)
    #r_value = 1 - np.sum(residuals**2) / np.sum(diff_mean**2)
    slope, intercept, r_value, p_value, std_err = stats.linregress(t_sec, values)

    corrections[s] = (slope, intercept, ci_slope_ppm)

    fig, ax = plt.subplots(figsize=(5.5, 3))
    ax.scatter(times, values, marker='o', linestyle='-', alpha=0.6)
    ax.plot(times, [intercept + slope * t for t in t_sec], 'r')
    ax.set_title(f"{s.name} : {intercept:.2f} + {slope*1e6:.3f}ppm (R² = {r_value**2:.2f}, uncertainty 95\% = {ci_slope_ppm:.2f}ppm)")
    ax.set_xlabel("Time")
    ax.set_ylabel("actual - expected (s)")
    ax.grid(True)
    ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1,4,7,10]))
    ax.xaxis.set_minor_locator(mdates.MonthLocator(bymonth=range(1, 13)))
    ax.xaxis.set_major_formatter(dates_format)

    res_csv += f'{s.name},{intercept:.2f},{slope*1e6:.3f},{ci_slope_ppm:.3f}\n'

    fig.patch.set_facecolor('xkcd:white')
    plt.savefig(f"../../../../data/MAHY/figures/dérives/{s.name}_drift.pdf",
    dpi=500,
    bbox_inches='tight',
    pad_inches=0
)

with open("../../../../data/detection/TiSSNet_Pn_raw_repicked/corrections.pkl", "wb") as f:
    pickle.dump(corrections, f)
with open("../../../../data/detection/TiSSNet_Pn_raw_repicked/corrections.csv", "w") as f:
    f.write(res_csv)

In [ ]:
# fixed intercept

corrections = {}  # contains, for each station, the slope and intercept of a linear correction (starting from s.date_start)
res_csv = ""

for i in range(1,5):
    local_stations = []
    time_series, time_series_s, value_series = {}, {}, {}

    for s in STATIONS:
        if s.name[-1] != str(i):
            continue
        if s.dataset not in match:
            print(f"Skipping station {s}")
            continue
        local_stations.append(s)

        values = np.array(list(deltas[s].values()))
        mask = np.abs(values - np.mean(values)) < 1*np.std(values)
        values = values[mask]
        times = np.array(list(deltas[s].keys()))[mask]
        value_series[s], time_series[s] = values[np.argsort(times)], times[np.argsort(times)]
        time_series_s[s] = [(t - s.date_start).total_seconds() for t in time_series[s]]

    total_points = sum(len(t) for t in time_series_s.values())
    X = np.zeros((total_points, len(local_stations) + 1))
    Y = np.zeros(total_points)
    row = 0
    for k, (t, y) in enumerate(zip(time_series_s.values(), value_series.values())):
        n_points = len(t)
        X[row:row+n_points, k] = t
        X[row:row+n_points, -1] = 1
        Y[row:row+n_points] = y
        row += n_points
    params, residuals, rank, s = np.linalg.lstsq(X, Y, rcond=None)
    n, p = X.shape
    sigma2 = SS_res / (n - p)
    cov_matrix = sigma2 * np.linalg.inv(X.T @ X)
    std_errors = np.sqrt(np.diag(cov_matrix))[:-1]
    uncertainties = 1.96 * std_errors * 1e6

    # PLOT
    intercept = params[-1]
    for si, s in enumerate(local_stations):
        slope = params[si]
        y_pred_k = slope * np.array(time_series_s[s]) + intercept
        ss_res_k = np.sum((value_series[s] - y_pred_k)**2)
        ss_tot_k = np.sum((value_series[s] - np.mean(y))**2)
        R2 = 1 - ss_res_k / ss_tot_k


        plt.figure(figsize=(12, 4))
        plt.scatter(time_series[s], value_series[s], marker='o', linestyle='-', alpha=0.6)
        plt.plot(time_series[s], [intercept + slope * t for t in time_series_s[s]], 'r')
        plt.title(f"{s.name} : {intercept:.2f} + {slope*1e6:.3f}ppm (R² = {R2:.2f}, uncertainty 95\% = {uncertainties[si]:.2f}ppm)")
        plt.xlabel("Time")
        plt.ylabel("actual - expected (s)")
        plt.grid(True)
        plt.tight_layout()

        res_csv += f'{s.name},{intercept:.2f},{slope*1e6:.3f},{uncertainties[si]:.3f}\n'
        corrections[s]=(slope, intercept)

        plt.savefig(f"../../../../data/MAHY/figures/drifts_fixed-intercept/{s.name}.png", dpi=500, bbox_inches='tight')


with open("../../../../data/detection/TiSSNet_Pn_raw_repicked/corrections_fixed-intercept.csv", "w") as f:
    f.write(res_csv)

In [ ]:
for s in STATIONS:
    detections = []
    with open(f"../../../../data/detection/TiSSNet_Pn_raw_repicked/{s.dataset}/{s.dataset}_{s.name}.pkl", "rb") as f:
        while True:
            try:
                detections.append(pickle.load(f))
            except EOFError:
                break
    new_detections = []
    for date, p in tqdm(detections[0]):
        delta = corrections[s][1] + corrections[s][0] * (date-s.date_start).total_seconds()
        new_detections.append((date - datetime.timedelta(seconds=delta), p))
    out_dir = f"../../../../data/detection/TiSSNet_Pn_raw_OBS-fixed/{s.dataset}"
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    with open(f"{out_dir}/{s.dataset}_{s.name}.pkl", "wb") as f:
        pickle.dump(np.array(new_detections), f)

In [ ]:
paths_to_chg = ["../../../../data/detection/TiSSNet_raw","../../../../data/detection/i_TiSSNet_raw"]

for path_to_chg in paths_to_chg:
    for s in STATIONS:
        detections = []
        with open(f"{path_to_chg}/{s.dataset}/{s.dataset}_{s.name}.pkl", "rb") as f:
            while True:
                try:
                    detections.append(pickle.load(f))
                except EOFError:
                    break
        new_detections = []
        for date, p in tqdm(np.array(detections).reshape(-1,2)):
            delta = corrections[s][0] * (date-s.date_start).total_seconds()
            new_detections.append((date - datetime.timedelta(seconds=delta), p))
        new_detections = np.array(new_detections)
        out_dir = f"{path_to_chg}_OBS-fixed/{s.dataset}"
        Path(out_dir).mkdir(parents=True, exist_ok=True)
        with open(f"{out_dir}/{s.dataset}_{s.name}.pkl", "wb") as f:
            pickle.dump(np.array(new_detections), f)

# Magnitudes

In [ ]:
mb = {}
RL = {}

DELTA = datetime.timedelta(seconds=1)

for dataset in match.keys():
    for idx in tqdm(match[dataset].index):
        a = match[dataset].loc[idx]
        for s in STATIONS.by_dataset(dataset):
            if not type(a[s.name]) == pd._libs.tslibs.nattype.NaTType:
                data = s.get_manager().get_segment(a[s.name] - DELTA, a[s.name] + DELTA)
                v = np.max(np.square(data))

                mb.setdefault(s, []).append(a["mb"])
                RL.setdefault(s, []).append(v)

In [ ]:
for s in STATIONS:
    slope, intercept, r_value, p_value, std_err = stats.linregress(mb[s], np.log10(RL[s]))

    plt.figure(figsize=(12, 4))

    plt.scatter(mb[s], np.log10(RL[s]), marker='o', linestyle='-', alpha=0.6)
    plt.plot(mb[s], intercept + slope * np.array(mb[s]), 'r')

    plt.title(f"{s.name} : {intercept:.2f} + {slope*1e6:.3f}ppm (R² = {r_value**2:.2f})")
    plt.xlabel("mb")
    plt.ylabel("RL (db re 1 uPa)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

### mb completeness

In [ ]:
S_df = pd.read_csv(
    "../../../../data/MAHY/lavayssiere_and_public.csv", header=None, names=["date","lat","lon","depth","mb"], parse_dates=["date"]
)

mb_detected = []
for dataset in match.keys():
    for idx in match[dataset].index:
        mb_detected.append(match[dataset].loc[idx]["mb"])
mb_detected = np.array(mb_detected)

m_list, v_list = [], []
for m in np.linspace(0,4,100):
    nb_events = np.count_nonzero(S_df["mb"] > m)
    nb_events_detected = np.count_nonzero(mb_detected > m)

    m_list.append(m)
    v_list.append(100 * nb_events_detected / nb_events)

plt.plot(m_list, v_list)